In [39]:
import tensorflow as tf
import tensorflow_datasets as tfds

In [10]:
df = tfds.load("wikipedia/20230601.simple")

In [38]:
df["train"].element_spec

{'text': TensorSpec(shape=(), dtype=tf.string, name=None),
 'title': TensorSpec(shape=(), dtype=tf.string, name=None)}